In [1]:
import open_clip
import torch

c:\Users\harma\Desktop\Price Discovery\Amazon product 2023\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="ViT-B-32", pretrained="laion2b_s34b_b79k"
)
tokenizer = open_clip.get_tokenizer(model_name="ViT-B-32")

In [3]:
import csv

with open("amazon_products.csv", "r", encoding="utf8") as file:
    lines = csv.reader(file)

    products = []
    metadata = []
    ids = []

    for i, line in enumerate(lines):
        if i == 0:
            continue
        products.append(line[1])
        metadata.append({"price": line[6]})
        ids.append(line[0])

In [5]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="clip_textdb")

In [7]:
# collection = chroma_client.create_collection(name="Text_Store")
collection = chroma_client.get_collection("Text_Store")

In [41]:
i, j = 80000, 82000
while j <= 100000:
    text = tokenizer(products[i:j])
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    emb = text_features.tolist()
    collection.add(
        documents=products[i:j], embeddings=emb, metadatas=metadata[i:j], ids=ids[i:j]
    )
    i = j
    j += 2000

In [29]:
# emb = text_features.tolist()
"""collection.add(
    documents = products[i:j],
    embeddings= emb,
    metadatas = metadata[i:j],
    ids = ids[i:j]
)"""

In [42]:
print(i, j)

100000 102000


In [43]:
collection.count()

100000

In [44]:
collection.peek()

{'ids': ['081298840X',
  '1319149588',
  '1338718568',
  '1417006048',
  '1435151720',
  '1473643147',
  '1610676238',
  '163884013X',
  '1640784748',
  '1733526102'],
 'embeddings': [[-0.049309808760881424,
   0.020820405334234238,
   -0.05177859961986542,
   0.0029519873205572367,
   -0.05800416320562363,
   -0.022951412945985794,
   -0.015298346057534218,
   -0.0562310665845871,
   -0.03265530616044998,
   -0.04772304743528366,
   0.04547961428761482,
   -0.018659744411706924,
   -0.015383000485599041,
   -0.02741990238428116,
   0.012051532045006752,
   -0.004783063195645809,
   -0.17041537165641785,
   0.03993026167154312,
   -0.05459268391132355,
   0.02414890192449093,
   0.036019135266542435,
   0.006192595232278109,
   0.036817241460084915,
   0.02445886842906475,
   0.016660913825035095,
   0.01648089289665222,
   0.02394028753042221,
   0.047255076467990875,
   0.010154355317354202,
   0.02659999206662178,
   -0.024528102949261665,
   -0.037694014608860016,
   0.060958191752

In [16]:
# chroma_client.delete_collection(name="Text_Store")